In [3732]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

import matplotlib.pyplot as plt
%matplotlib inline

In [3733]:
import csv
with open('flights_train.csv') as f:
    reader = csv.reader(f)
    h_names = next(reader)
h_names

['UID',
 'DAY_OF_WEEK',
 'CRS_DEP_TIME',
 'TAXI_OUT',
 'TAXI_IN',
 'ACTUAL_ELAPSED_TIME',
 'DISTANCE',
 'DISTANCE_GROUP',
 'AIRLINE_ID',
 'FL_DATE',
 'ORIGIN_CITY_MARKET_ID',
 'DEST_CITY_MARKET_ID',
 'FL_NUM',
 'UNIQUE_CARRIER',
 'ORIGIN',
 'ORIGIN_CITY_NAME',
 'ORIGIN_STATE_ABR',
 'DEST',
 'DEST_CITY_NAME',
 'DEST_STATE_ABR',
 'FIRST_DEP_TIME',
 'ARR_DELAY']

# importing test data

In [3734]:
df_test = pd.read_csv('flights_test.csv')

In [3735]:
df_test.head(5)

,UID,DAY_OF_WEEK,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,AIRLINE_ID,FL_DATE,...,DEST_CITY_MARKET_ID,FL_NUM,UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_CITY_NAME,DEST_STATE_ABR,FIRST_DEP_TIME
0,8,3,2000,18,5,83,337,2,20304,2017-01-25,...,32457,4926,OO,LAX,"Los Angeles, CA",CA,SFO,"San Francisco, CA",CA,NaN
1,15,2,1830,12,4,182,"1,476",6,19393,2017-01-10,...,30423,336,WN,SJC,"San Jose, CA",CA,AUS,"Austin, TX",TX,NaN
2,29,2,1519,17,4,138,780,4,20366,2017-01-17,...,30397,5146,EV,HPN,"White Plains, NY",NY,ATL,"Atlanta, GA",GA,NaN
3,33,1,940,10,2,116,773,4,19790,2017-01-09,...,33195,1502,DL,CVG,"Cincinnati, OH",KY,TPA,"Tampa, FL",FL,NaN
4,65,6,1425,19,5,278,"2,419",10,21171,2017-01-07,...,30852,90,VX,SFO,"San Francisco, CA",CA,IAD,"Washington, DC",VA,NaN


# importing training data

In [3736]:
df = pd.read_csv('flights_train.csv')
df.head(5)


,UID,DAY_OF_WEEK,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,AIRLINE_ID,FL_DATE,...,FL_NUM,UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_CITY_NAME,DEST_STATE_ABR,FIRST_DEP_TIME,ARR_DELAY
0,0,2,1020,11,8,373,"2,701",11,19930,2017-01-03,...,879,AS,SEA,"Seattle, WA",WA,LIH,"Lihue, HI",HI,NaN,-11
1,1,6,1220,13,9,183,"1,171",5,19805,2017-01-28,...,2491,AA,SAN,"San Diego, CA",CA,DFW,"Dallas/Fort Worth, TX",TX,NaN,1
2,2,6,945,48,6,355,"2,552",11,19930,2017-01-21,...,172,AS,ANC,"Anchorage, AK",AK,PHX,"Phoenix, AZ",AZ,NaN,18
3,3,1,1437,17,8,136,967,4,19977,2017-01-23,...,2030,UA,SFO,"San Francisco, CA",CA,DEN,"Denver, CO",CO,NaN,0
4,4,1,830,37,7,114,534,3,19393,2017-01-02,...,3812,WN,ATL,"Atlanta, GA",GA,IAD,"Washington, DC",VA,NaN,197


# Checking Missing Values

In [3737]:
# we check which of the header_names have missing values.
for nam in h_names:
    a = df[[nam]].isnull().values.any()
    print(nam, a)

UID False
DAY_OF_WEEK False
CRS_DEP_TIME False
TAXI_OUT False
TAXI_IN False
ACTUAL_ELAPSED_TIME False
DISTANCE False
DISTANCE_GROUP False
AIRLINE_ID False
FL_DATE False
ORIGIN_CITY_MARKET_ID False
DEST_CITY_MARKET_ID False
FL_NUM False
UNIQUE_CARRIER False
ORIGIN False
ORIGIN_CITY_NAME False
ORIGIN_STATE_ABR False
DEST False
DEST_CITY_NAME False
DEST_STATE_ABR False
FIRST_DEP_TIME True
ARR_DELAY False


In [3738]:
#we delete the column First_Dep_time
df = df.drop('FIRST_DEP_TIME', 1)
df.head()


,UID,DAY_OF_WEEK,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,AIRLINE_ID,FL_DATE,...,DEST_CITY_MARKET_ID,FL_NUM,UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_CITY_NAME,DEST_STATE_ABR,ARR_DELAY
0,0,2,1020,11,8,373,"2,701",11,19930,2017-01-03,...,32982,879,AS,SEA,"Seattle, WA",WA,LIH,"Lihue, HI",HI,-11
1,1,6,1220,13,9,183,"1,171",5,19805,2017-01-28,...,30194,2491,AA,SAN,"San Diego, CA",CA,DFW,"Dallas/Fort Worth, TX",TX,1
2,2,6,945,48,6,355,"2,552",11,19930,2017-01-21,...,30466,172,AS,ANC,"Anchorage, AK",AK,PHX,"Phoenix, AZ",AZ,18
3,3,1,1437,17,8,136,967,4,19977,2017-01-23,...,30325,2030,UA,SFO,"San Francisco, CA",CA,DEN,"Denver, CO",CO,0
4,4,1,830,37,7,114,534,3,19393,2017-01-02,...,30852,3812,WN,ATL,"Atlanta, GA",GA,IAD,"Washington, DC",VA,197


In [3739]:
#we delete the column First_Dep_time in test data as well.
df_test = df_test.drop('FIRST_DEP_TIME', 1)
df_test.head(3)

,UID,DAY_OF_WEEK,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,AIRLINE_ID,FL_DATE,ORIGIN_CITY_MARKET_ID,DEST_CITY_MARKET_ID,FL_NUM,UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_CITY_NAME,DEST_STATE_ABR
0,8,3,2000,18,5,83,337,2,20304,2017-01-25,32575,32457,4926,OO,LAX,"Los Angeles, CA",CA,SFO,"San Francisco, CA",CA
1,15,2,1830,12,4,182,"1,476",6,19393,2017-01-10,32457,30423,336,WN,SJC,"San Jose, CA",CA,AUS,"Austin, TX",TX
2,29,2,1519,17,4,138,780,4,20366,2017-01-17,31703,30397,5146,EV,HPN,"White Plains, NY",NY,ATL,"Atlanta, GA",GA


In [3740]:
# CHECKING FREQ. OF EACH FEATURE
# df.apply(pd.value_counts)
# df.apply(pd.value_counts).fillna(0)
kk ='ORIGIN'
print (df[kk].value_counts())
print('length:')
print(len(df[kk].value_counts()))

ATL    317
DEN    205
ORD    200
LAX    189
SFO    164
DFW    146
LAS    139
DTW    124
PHX    123
BOS    121
SEA    120
SLC    120
MSP    118
MCO    106
IAH    104
EWR     92
FLL     92
JFK     91
CLT     82
BWI     79
MDW     78
LGA     71
MIA     66
DCA     63
TPA     63
DAL     61
PDX     57
SAN     53
PHL     52
MSY     52
      ... 
BTM      1
MHK      1
YAK      1
GNV      1
FSM      1
BRW      1
GPT      1
FAY      1
OAJ      1
HDN      1
CSG      1
PIB      1
MOT      1
FAI      1
MFR      1
AVP      1
SUN      1
AGS      1
BRO      1
SGF      1
RST      1
PGD      1
BLI      1
ISN      1
TOL      1
STX      1
SCE      1
DRO      1
HLN      1
CHO      1
Name: ORIGIN, Length: 239, dtype: int64
length:
239


# Dealing with the outliers
#cut off value is 150

In [3741]:
outl = df['ARR_DELAY'] < 300
df_new = df[outl]
df=df_new
print('length:')
print(len(df))

length:
4891


In [3742]:
# Number of examples left after removing the outliers
df.shape

(4891, 21)

In [3743]:
df_test.shape

(484, 20)

# Dealing with individual Features

### one hot encoding


In [3744]:

df_train =df
one_hot = pd.get_dummies(df_train['DAY_OF_WEEK'])
# delete the old feature
df_train = df_train.drop('DAY_OF_WEEK',axis=1)
df_train = df_train.join(one_hot)
# one_hot.head()
df_train.head(5)
print(df_train.shape)
df_train.head(5)




(4891, 27)


,UID,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,AIRLINE_ID,FL_DATE,ORIGIN_CITY_MARKET_ID,...,DEST_CITY_NAME,DEST_STATE_ABR,ARR_DELAY,1,2,3,4,5,6,7
0,0,1020,11,8,373,"2,701",11,19930,2017-01-03,30559,...,"Lihue, HI",HI,-11,0,1,0,0,0,0,0
1,1,1220,13,9,183,"1,171",5,19805,2017-01-28,33570,...,"Dallas/Fort Worth, TX",TX,1,0,0,0,0,0,1,0
2,2,945,48,6,355,"2,552",11,19930,2017-01-21,30299,...,"Phoenix, AZ",AZ,18,0,0,0,0,0,1,0
3,3,1437,17,8,136,967,4,19977,2017-01-23,32457,...,"Denver, CO",CO,0,1,0,0,0,0,0,0
4,4,830,37,7,114,534,3,19393,2017-01-02,30397,...,"Washington, DC",VA,197,1,0,0,0,0,0,0


In [3745]:
# do the same for df_test
one_hot_test = pd.get_dummies(df_test['DAY_OF_WEEK'])
# delete the old feature
df_test = df_test.drop('DAY_OF_WEEK',axis=1)
df_test = df_test.join(one_hot_test)
# one_hot.head()
df_test.head(5)
print(df_test.shape)
df_test.head(5)

(484, 26)


,UID,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,AIRLINE_ID,FL_DATE,ORIGIN_CITY_MARKET_ID,...,DEST,DEST_CITY_NAME,DEST_STATE_ABR,1,2,3,4,5,6,7
0,8,2000,18,5,83,337,2,20304,2017-01-25,32575,...,SFO,"San Francisco, CA",CA,0,0,1,0,0,0,0
1,15,1830,12,4,182,"1,476",6,19393,2017-01-10,32457,...,AUS,"Austin, TX",TX,0,1,0,0,0,0,0
2,29,1519,17,4,138,780,4,20366,2017-01-17,31703,...,ATL,"Atlanta, GA",GA,0,1,0,0,0,0,0
3,33,940,10,2,116,773,4,19790,2017-01-09,33105,...,TPA,"Tampa, FL",FL,1,0,0,0,0,0,0
4,65,1425,19,5,278,"2,419",10,21171,2017-01-07,32457,...,IAD,"Washington, DC",VA,0,0,0,0,0,1,0


In [3746]:
# renaming the columns for day of week-  hot encoded
df_train = df_train.rename(columns={1: 'Mon', 2: 'Tue',3: 'Wed',4: 'Thurs',5: 'Fri',6: 'Sat',7: 'Sun'})
df_train.head(2)
column_names = df_train.columns.values
print(column_names)


['UID' 'CRS_DEP_TIME' 'TAXI_OUT' 'TAXI_IN' 'ACTUAL_ELAPSED_TIME'
 'DISTANCE' 'DISTANCE_GROUP' 'AIRLINE_ID' 'FL_DATE'
 'ORIGIN_CITY_MARKET_ID' 'DEST_CITY_MARKET_ID' 'FL_NUM' 'UNIQUE_CARRIER'
 'ORIGIN' 'ORIGIN_CITY_NAME' 'ORIGIN_STATE_ABR' 'DEST' 'DEST_CITY_NAME'
 'DEST_STATE_ABR' 'ARR_DELAY' 'Mon' 'Tue' 'Wed' 'Thurs' 'Fri' 'Sat' 'Sun']


In [3747]:
# do the same for df_test
df_test = df_test.rename(columns={1: 'Mon', 2: 'Tue',3: 'Wed',4: 'Thurs',5: 'Fri',6: 'Sat',7: 'Sun'})
df_test.head(2)
column_names = df_test.columns.values
print(column_names)

['UID' 'CRS_DEP_TIME' 'TAXI_OUT' 'TAXI_IN' 'ACTUAL_ELAPSED_TIME'
 'DISTANCE' 'DISTANCE_GROUP' 'AIRLINE_ID' 'FL_DATE'
 'ORIGIN_CITY_MARKET_ID' 'DEST_CITY_MARKET_ID' 'FL_NUM' 'UNIQUE_CARRIER'
 'ORIGIN' 'ORIGIN_CITY_NAME' 'ORIGIN_STATE_ABR' 'DEST' 'DEST_CITY_NAME'
 'DEST_STATE_ABR' 'Mon' 'Tue' 'Wed' 'Thurs' 'Fri' 'Sat' 'Sun']


# HOT ENCODING THE BELOW FEATURES

In [3748]:
arr_one_hot = ['AIRLINE_ID',
               'UNIQUE_CARRIER'
               ]
# hot_endoed orgin state 
#
for x in arr_one_hot:
    one_hot = pd.get_dummies(df_train[x])
    df_train = df_train.drop(x,1)
    df_train = df_train.join(one_hot)
    print(x)
    print(df_train.shape)

df_train.head(5)

AIRLINE_ID
(4891, 38)
UNIQUE_CARRIER
(4891, 49)


,UID,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,FL_DATE,ORIGIN_CITY_MARKET_ID,DEST_CITY_MARKET_ID,...,B6,DL,EV,F9,HA,NK,OO,UA,VX,WN
0,0,1020,11,8,373,"2,701",11,2017-01-03,30559,32982,...,0,0,0,0,0,0,0,0,0,0
1,1,1220,13,9,183,"1,171",5,2017-01-28,33570,30194,...,0,0,0,0,0,0,0,0,0,0
2,2,945,48,6,355,"2,552",11,2017-01-21,30299,30466,...,0,0,0,0,0,0,0,0,0,0
3,3,1437,17,8,136,967,4,2017-01-23,32457,30325,...,0,0,0,0,0,0,0,1,0,0
4,4,830,37,7,114,534,3,2017-01-02,30397,30852,...,0,0,0,0,0,0,0,0,0,1


In [3749]:
#do the same for df_test
arr_one_hot_test = ['AIRLINE_ID',
               'UNIQUE_CARRIER'
              ]
for x in arr_one_hot_test:
    one_hot_test = pd.get_dummies(df_test[x])
    df_test = df_test.drop(x,1)
    df_test = df_test.join(one_hot_test)
    print(x)
    print(df_test.shape)

df_test.head(5)

AIRLINE_ID
(484, 37)
UNIQUE_CARRIER
(484, 48)


,UID,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,FL_DATE,ORIGIN_CITY_MARKET_ID,DEST_CITY_MARKET_ID,...,B6,DL,EV,F9,HA,NK,OO,UA,VX,WN
0,8,2000,18,5,83,337,2,2017-01-25,32575,32457,...,0,0,0,0,0,0,1,0,0,0
1,15,1830,12,4,182,"1,476",6,2017-01-10,32457,30423,...,0,0,0,0,0,0,0,0,0,1
2,29,1519,17,4,138,780,4,2017-01-17,31703,30397,...,0,0,1,0,0,0,0,0,0,0
3,33,940,10,2,116,773,4,2017-01-09,33105,33195,...,0,1,0,0,0,0,0,0,0,0
4,65,1425,19,5,278,"2,419",10,2017-01-07,32457,30852,...,0,0,0,0,0,0,0,0,1,0


In [3750]:
df_train.head(5)


,UID,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,FL_DATE,ORIGIN_CITY_MARKET_ID,DEST_CITY_MARKET_ID,...,B6,DL,EV,F9,HA,NK,OO,UA,VX,WN
0,0,1020,11,8,373,"2,701",11,2017-01-03,30559,32982,...,0,0,0,0,0,0,0,0,0,0
1,1,1220,13,9,183,"1,171",5,2017-01-28,33570,30194,...,0,0,0,0,0,0,0,0,0,0
2,2,945,48,6,355,"2,552",11,2017-01-21,30299,30466,...,0,0,0,0,0,0,0,0,0,0
3,3,1437,17,8,136,967,4,2017-01-23,32457,30325,...,0,0,0,0,0,0,0,1,0,0
4,4,830,37,7,114,534,3,2017-01-02,30397,30852,...,0,0,0,0,0,0,0,0,0,1


### ALSO NEED TO HOT ENCODE DEST_STATE_ABR. FOR NOW DELETING IT!!!########################################3

#### CHANGING THE NAME OF STATES FOR ORIGIN STATES,
### LEAVE IT NOW

In [3751]:
# df_orig= pd.read_csv('flights_train.csv')
# df_orig['ORIG_']
# df_train = df_train.rename(columns={1: 'x'+str(i)})

# #change the feature FL-DATE to 'month'


In [3752]:
df1= df_train
df1['month'] = pd.DatetimeIndex(df1['FL_DATE']).month
column_names = df1.columns.values
print(column_names)
print(df1.shape)

# print(df1['month'])
# df1['month']

# do same for test
df2= df_test
df2['month'] = pd.DatetimeIndex(df2['FL_DATE']).month
column_names = df2.columns.values
print(column_names)
print(df2.shape)


['UID' 'CRS_DEP_TIME' 'TAXI_OUT' 'TAXI_IN' 'ACTUAL_ELAPSED_TIME'
 'DISTANCE' 'DISTANCE_GROUP' 'FL_DATE' 'ORIGIN_CITY_MARKET_ID'
 'DEST_CITY_MARKET_ID' 'FL_NUM' 'ORIGIN' 'ORIGIN_CITY_NAME'
 'ORIGIN_STATE_ABR' 'DEST' 'DEST_CITY_NAME' 'DEST_STATE_ABR' 'ARR_DELAY'
 'Mon' 'Tue' 'Wed' 'Thurs' 'Fri' 'Sat' 'Sun' 19393 19690 19790 19805 19930
 19977 20304 20366 20409 20416 20436 21171 'AA' 'AS' 'B6' 'DL' 'EV' 'F9'
 'HA' 'NK' 'OO' 'UA' 'VX' 'WN' 'month']
(4891, 50)
['UID' 'CRS_DEP_TIME' 'TAXI_OUT' 'TAXI_IN' 'ACTUAL_ELAPSED_TIME'
 'DISTANCE' 'DISTANCE_GROUP' 'FL_DATE' 'ORIGIN_CITY_MARKET_ID'
 'DEST_CITY_MARKET_ID' 'FL_NUM' 'ORIGIN' 'ORIGIN_CITY_NAME'
 'ORIGIN_STATE_ABR' 'DEST' 'DEST_CITY_NAME' 'DEST_STATE_ABR' 'Mon' 'Tue'
 'Wed' 'Thurs' 'Fri' 'Sat' 'Sun' 19393 19690 19790 19805 19930 19977 20304
 20366 20409 20416 20436 21171 'AA' 'AS' 'B6' 'DL' 'EV' 'F9' 'HA' 'NK'
 'OO' 'UA' 'VX' 'WN' 'month']
(484, 49)


In [3753]:
# convert month to hot coding
one_hot = pd.get_dummies(df1['month'])
df1 = df1.drop('month',axis=1)
df1 = df1.join(one_hot)
# one_hot.head()
df1.head(5)
print(df_train.shape)
df1.head(5)



# do same for df_test
one_hot_test = pd.get_dummies(df2['month'])
df2 = df2.drop('month',axis=1)
df2 = df2.join(one_hot_test)
# one_hot.head()
df2.head(5)
print(df2.shape)
df2.head(5)

(4891, 50)
(484, 60)


,UID,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,FL_DATE,ORIGIN_CITY_MARKET_ID,DEST_CITY_MARKET_ID,...,3,4,5,6,7,8,9,10,11,12
0,8,2000,18,5,83,337,2,2017-01-25,32575,32457,...,0,0,0,0,0,0,0,0,0,0
1,15,1830,12,4,182,"1,476",6,2017-01-10,32457,30423,...,0,0,0,0,0,0,0,0,0,0
2,29,1519,17,4,138,780,4,2017-01-17,31703,30397,...,0,0,0,0,0,0,0,0,0,0
3,33,940,10,2,116,773,4,2017-01-09,33105,33195,...,0,0,0,0,0,0,0,0,0,0
4,65,1425,19,5,278,"2,419",10,2017-01-07,32457,30852,...,0,0,0,0,0,0,0,0,0,0


#### NICE CODE FOR COLUMN NAMES

In [3754]:
column_names = df1.columns.values
print(column_names)
print("Number of att: ", len(column_names))
print(df1.shape)

column_names = df2.columns.values
print(column_names)
print("Number of att: ", len(column_names))
print(df2.shape)

['UID' 'CRS_DEP_TIME' 'TAXI_OUT' 'TAXI_IN' 'ACTUAL_ELAPSED_TIME'
 'DISTANCE' 'DISTANCE_GROUP' 'FL_DATE' 'ORIGIN_CITY_MARKET_ID'
 'DEST_CITY_MARKET_ID' 'FL_NUM' 'ORIGIN' 'ORIGIN_CITY_NAME'
 'ORIGIN_STATE_ABR' 'DEST' 'DEST_CITY_NAME' 'DEST_STATE_ABR' 'ARR_DELAY'
 'Mon' 'Tue' 'Wed' 'Thurs' 'Fri' 'Sat' 'Sun' 19393 19690 19790 19805 19930
 19977 20304 20366 20409 20416 20436 21171 'AA' 'AS' 'B6' 'DL' 'EV' 'F9'
 'HA' 'NK' 'OO' 'UA' 'VX' 'WN' 1 2 3 4 5 6 7 8 9 10 11 12]
Number of att:  61
(4891, 61)
['UID' 'CRS_DEP_TIME' 'TAXI_OUT' 'TAXI_IN' 'ACTUAL_ELAPSED_TIME'
 'DISTANCE' 'DISTANCE_GROUP' 'FL_DATE' 'ORIGIN_CITY_MARKET_ID'
 'DEST_CITY_MARKET_ID' 'FL_NUM' 'ORIGIN' 'ORIGIN_CITY_NAME'
 'ORIGIN_STATE_ABR' 'DEST' 'DEST_CITY_NAME' 'DEST_STATE_ABR' 'Mon' 'Tue'
 'Wed' 'Thurs' 'Fri' 'Sat' 'Sun' 19393 19690 19790 19805 19930 19977 20304
 20366 20409 20416 20436 21171 'AA' 'AS' 'B6' 'DL' 'EV' 'F9' 'HA' 'NK'
 'OO' 'UA' 'VX' 'WN' 1 2 3 4 5 6 7 8 9 10 11 12]
Number of att:  60
(484, 60)


# DELETING SOME FEATURES 

In [3755]:
df =df1

In [3756]:


# Note that we also delete FL_DATE
remov_att = ['FL_NUM',
             'FL_DATE',
               'ORIGIN',  
               'ORIGIN_CITY_MARKET_ID',
               'ORIGIN_CITY_NAME',
               'DEST_CITY_MARKET_ID',
               'DEST',
               'DEST_CITY_NAME',
               'CRS_DEP_TIME',
               'DISTANCE',
              'DISTANCE_GROUP',
            'DEST_STATE_ABR',
            'ORIGIN_STATE_ABR'
            ]
for x in remov_att:
    df = df.drop(x,1)
    print(x)
    print(df.shape)

df.head(5)



FL_NUM
(4891, 60)
FL_DATE
(4891, 59)
ORIGIN
(4891, 58)
ORIGIN_CITY_MARKET_ID
(4891, 57)
ORIGIN_CITY_NAME
(4891, 56)
DEST_CITY_MARKET_ID
(4891, 55)
DEST
(4891, 54)
DEST_CITY_NAME
(4891, 53)
CRS_DEP_TIME
(4891, 52)
DISTANCE
(4891, 51)
DISTANCE_GROUP
(4891, 50)
DEST_STATE_ABR
(4891, 49)
ORIGIN_STATE_ABR
(4891, 48)


,UID,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,ARR_DELAY,Mon,Tue,Wed,Thurs,Fri,...,3,4,5,6,7,8,9,10,11,12
0,0,11,8,373,-11,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,13,9,183,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,48,6,355,18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,17,8,136,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,37,7,114,197,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3757]:
df_test =df2

In [3758]:
# do the same for df_test
# Note that we also delete FL_DATE
remov_att = ['FL_NUM',
             'FL_DATE',
               'ORIGIN',  
               'ORIGIN_CITY_MARKET_ID',
               'ORIGIN_CITY_NAME',
               'DEST_CITY_MARKET_ID',
               'DEST',
               'DEST_CITY_NAME',
               'CRS_DEP_TIME',
               'DISTANCE',
              'DISTANCE_GROUP',
            'DEST_STATE_ABR',
            'ORIGIN_STATE_ABR'
            ]
for x in remov_att:
    df_test = df_test.drop(x,1)
    print(x)
    print(df_test.shape)

df_test.head(5)


FL_NUM
(484, 59)
FL_DATE
(484, 58)
ORIGIN
(484, 57)
ORIGIN_CITY_MARKET_ID
(484, 56)
ORIGIN_CITY_NAME
(484, 55)
DEST_CITY_MARKET_ID
(484, 54)
DEST
(484, 53)
DEST_CITY_NAME
(484, 52)
CRS_DEP_TIME
(484, 51)
DISTANCE
(484, 50)
DISTANCE_GROUP
(484, 49)
DEST_STATE_ABR
(484, 48)
ORIGIN_STATE_ABR
(484, 47)


,UID,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,Mon,Tue,Wed,Thurs,Fri,Sat,...,3,4,5,6,7,8,9,10,11,12
0,8,18,5,83,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15,12,4,182,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,29,17,4,138,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,33,10,2,116,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,65,19,5,278,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## VISUALIZE THE DATA

In [3759]:
# print("PRINTING TRAINING DATA:")
# column_names = df.columns.values
# print(column_names)
# print("Number of att: ", len(column_names))
# print(df.iloc[:, 0:18].head(3))
# print(df.iloc[:, 18:36].head(3))
# print(df.iloc[:, 36:54].head(3))
# print(df.iloc[:, 54:72].head(3))
# print(df.iloc[:, 72:90].head(3))
# print(df.iloc[:, 90:108].head(3))
# # print(df.iloc[:, 0:18].head(3))
# print(df.shape)

In [3760]:
# print("PRINTING TEST DATA:")
# column_names = df_test.columns.values
# print(column_names)
# print("Number of att: ", len(column_names))
# print(df.iloc[:, 0:18].head(3))
# print(df.iloc[:, 18:36].head(3))
# print(df.iloc[:, 36:54].head(3))
# print(df.iloc[:, 54:72].head(3))
# print(df.iloc[:, 72:90].head(3))
# print(df.iloc[:, 90:108].head(3))
# # print(df.iloc[:, 0:18].head(3))
# print(df_test.shape)

# Converting Data to numpy for DNN

In [3761]:
x_train_df = df.drop('ARR_DELAY',axis=1)
x_train_df = x_train_df.drop('UID',axis=1)
r_train_df = df[['ARR_DELAY']]


In [3762]:
x_test_df = df_test
x_test_df = x_test_df.drop('UID',axis=1)


In [3763]:
print(x_train_df.shape)
print(r_train_df.shape)
print(x_test_df.shape)

(4891, 46)
(4891, 1)
(484, 46)


In [3764]:
print("PRINTING TRAINING DATA:")
column_names = x_train_df.columns.values
print(column_names)
print("Number of att: ", len(column_names))
print(x_train_df.iloc[:, 0:18].head(3))
print(x_train_df.iloc[:, 18:36].head(3))
print(x_train_df.iloc[:, 36:54].head(3))
print(x_train_df.iloc[:, 54:72].head(3))
print(x_train_df.iloc[:, 72:90].head(3))
print(x_train_df.iloc[:, 90:108].head(3))
# print(df.iloc[:, 0:18].head(3))
print(x_train_df.shape)

PRINTING TRAINING DATA:
['TAXI_OUT' 'TAXI_IN' 'ACTUAL_ELAPSED_TIME' 'Mon' 'Tue' 'Wed' 'Thurs'
 'Fri' 'Sat' 'Sun' 19393 19690 19790 19805 19930 19977 20304 20366 20409
 20416 20436 21171 'AA' 'AS' 'B6' 'DL' 'EV' 'F9' 'HA' 'NK' 'OO' 'UA' 'VX'
 'WN' 1 2 3 4 5 6 7 8 9 10 11 12]
Number of att:  46
   TAXI_OUT  TAXI_IN  ACTUAL_ELAPSED_TIME  Mon  Tue  Wed  Thurs  Fri  Sat  \
0        11        8                  373    0    1    0      0    0    0   
1        13        9                  183    0    0    0      0    0    1   
2        48        6                  355    0    0    0      0    0    1   

   Sun  19393  19690  19790  19805  19930  19977  20304  20366  
0    0      0      0      0      0      1      0      0      0  
1    0      0      0      0      1      0      0      0      0  
2    0      0      0      0      0      1      0      0      0  
   20409  20416  20436  21171  AA  AS  B6  DL  EV  F9  HA  NK  OO  UA  VX  WN  \
0      0      0      0      0   0   1   0   0   0   0   

In [3765]:
print("PRINTING TEST DATA:")
column_names = x_test_df.columns.values
print(column_names)
print("Number of att: ", len(column_names))
print(x_test_df.iloc[:, 0:18].head(3))
print(x_test_df.iloc[:, 18:36].head(3))
print(x_test_df.iloc[:, 36:54].head(3))
print(x_test_df.iloc[:, 54:72].head(3))
print(x_test_df.iloc[:, 72:90].head(3))
print(x_test_df.iloc[:, 90:108].head(3))
# print(df.iloc[:, 0:18].head(3))
print(x_test_df.shape)

PRINTING TEST DATA:
['TAXI_OUT' 'TAXI_IN' 'ACTUAL_ELAPSED_TIME' 'Mon' 'Tue' 'Wed' 'Thurs'
 'Fri' 'Sat' 'Sun' 19393 19690 19790 19805 19930 19977 20304 20366 20409
 20416 20436 21171 'AA' 'AS' 'B6' 'DL' 'EV' 'F9' 'HA' 'NK' 'OO' 'UA' 'VX'
 'WN' 1 2 3 4 5 6 7 8 9 10 11 12]
Number of att:  46
   TAXI_OUT  TAXI_IN  ACTUAL_ELAPSED_TIME  Mon  Tue  Wed  Thurs  Fri  Sat  \
0        18        5                   83    0    0    1      0    0    0   
1        12        4                  182    0    1    0      0    0    0   
2        17        4                  138    0    1    0      0    0    0   

   Sun  19393  19690  19790  19805  19930  19977  20304  20366  
0    0      0      0      0      0      0      0      1      0  
1    0      1      0      0      0      0      0      0      0  
2    0      0      0      0      0      0      0      0      1  
   20409  20416  20436  21171  AA  AS  B6  DL  EV  F9  HA  NK  OO  UA  VX  WN  \
0      0      0      0      0   0   0   0   0   0   0   0   

In [3766]:
# normazling
# import sklearn
# x_train = x_train_df.values
# x_train = sklearn.preprocessing.normalize(x_train)
# x_train.shape
# print(x_train)
x_train = x_train_df.values
x_train.shape

(4891, 46)

In [3767]:
r_train = r_train_df.values
y_train=r_train
# y_train= y_train.transpose()
r_train.shape


(4891, 1)

In [3768]:
x_test_df = x_test_df.values
x_test_df.shape

(484, 46)

# PCA 

In [3769]:
# import sklearn.decomposition as skd

# # .fit computes the principal components  (n_components of them)
# # The columns of W are the eigenvectors of the covariance matrix of X
# pca = skd.PCA(n_components = 20)
# skd.PCA.fit(pca,x_train)
# W1 = pca.components_
# W = W1.transpose()
# Z = pca.transform(x_train)
# print(W)
# print(Z)


In [3770]:
# x_train=Z
# x_train.shape

In [3771]:
# test_inputs = torch.from_numpy(x_train)
# test_targets = torch.from_numpy(y_train)
# test_inputs

In [3772]:
# Toy dataset
# x_train = np.array([[3.3], [4.4], [5.5], [6.71], [6.93], [4.168], 
#                     [9.779], [6.182], [7.59], [2.167], [7.042], 
#                     [10.791], [5.313], [7.997], [3.1]], dtype=np.float32)

# y_train = np.array([[1.7], [2.76], [2.09], [3.19], [1.694], [1.573], 
#                     [3.366], [2.596], [2.53], [1.221], [2.827], 
#                     [3.465], [1.65], [2.904], [1.3]], dtype=np.float32)
# print(x_train.shape)
# print(y_train.shape)

In [3773]:
# Toy dataset
# x_train = np.array([[3.3,1,1], [4.4,1,1], [5.5,2,2], [6.71,2,2], [6.93,2,2]], dtype=np.float32)

# y_train = np.array([[1.7], [2.76], [2.09], [3.19], [1.694]], dtype=np.float32)


# Fully-Connected DNN

In [3774]:
"""
View more, visit my tutorial page: https://morvanzhou.github.io/tutorials/
My Youtube Channel: https://www.youtube.com/user/MorvanZhou
Dependencies:
torch: 0.1.11
matplotlib
"""
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt




class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer # n_feature is input layer inputs,output layer
        # nodes are the n_hidden
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x
    




# Fully-Connected DNN

In [3775]:
# class Net(torch.nn.Module):
#     def __init__(self, n_feature, n_hidden_1,n_hidden_2, n_output):
#         super(Net, self).__init__()
#         self.hidden_a = torch.nn.Linear(n_feature, n_hidden_1)   # hidden layer # n_feature is input layer inputs,output layer
#         # nodes are the n_hidden
        
#         self.hidden_b = torch.nn.Linear(n_hidden_1, n_hidden_2)
        
#         self.predict = torch.nn.Linear(n_hidden_2, n_output)   # output layer

#     def forward(self, x):
#         x = F.tanh(self.hidden_a(x))      # activation function for hidden layer
#         x = F.tanh(self.hidden_b(x))
#         x = self.predict(x)             # linear output
#         return x

# Defining the NN, Loss Function and Optimizer

In [3779]:
model = Net(n_feature=46, n_hidden=15, n_output=1)     # define the network
print(model)  # net architecture

# optimizer = torch.optim.SGD(net.parameters(), lr=0.5, momentum=0.9)
# loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

Net(
  (hidden): Linear(in_features=46, out_features=15, bias=True)
  (predict): Linear(in_features=15, out_features=1, bias=True)
)


In [3780]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt


# Hyper-parameters
num_epochs = 200
learning_rate = 0.001

# # Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  



# x_train = np.array([[3.3,1,1], [4.4,1,1], [5.5,2,2], [6.71,2,2], [6.93,2,2]], dtype=np.float32)

# y_train = np.array([[1.7], [2.76], [2.09], [3.19], [1.694]], dtype=np.float32)

# Train the model
for epoch in range(num_epochs):
    # Convert numpy arrays to torch tensors
#     inputs = torch.from_numpy(x_train)
#     targets = torch.from_numpy(y_train)
    
    inputs = Variable(torch.from_numpy(x_train).float(), requires_grad=False)
    targets = Variable(torch.from_numpy(y_train).float(), requires_grad=False)

    # Forward pass
    outputs = model(inputs)
    # input x and predict based on x - prediction is the outputs
    loss = criterion(outputs, targets)
     # must be (1. nn output, 2. target)
    
    # Backward and optimize
    optimizer.zero_grad()  # clear gradients for next train
    loss.backward() # backpropagation, compute gradients
    optimizer.step()  # apply gradients
 
         

    
    if (epoch+1) % 5 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))




Epoch [5/200], Loss: 1173.6202
Epoch [10/200], Loss: 1173.4976
Epoch [15/200], Loss: 1173.3733
Epoch [20/200], Loss: 1173.2997
Epoch [25/200], Loss: 1173.2048
Epoch [30/200], Loss: 1173.1061
Epoch [35/200], Loss: 1173.0184
Epoch [40/200], Loss: 1172.9248
Epoch [45/200], Loss: 1172.8402
Epoch [50/200], Loss: 1172.7523
Epoch [55/200], Loss: 1172.6689
Epoch [60/200], Loss: 1172.5820
Epoch [65/200], Loss: 1172.5031
Epoch [70/200], Loss: 1172.4254
Epoch [75/200], Loss: 1172.3613
Epoch [80/200], Loss: 1172.2635
Epoch [85/200], Loss: 1172.1890
Epoch [90/200], Loss: 1172.1172
Epoch [95/200], Loss: 1172.0492
Epoch [100/200], Loss: 1171.9791
Epoch [105/200], Loss: 1171.9052
Epoch [110/200], Loss: 1171.8400
Epoch [115/200], Loss: 1171.7676
Epoch [120/200], Loss: 1171.7035
Epoch [125/200], Loss: 1171.6418
Epoch [130/200], Loss: 1171.5792
Epoch [135/200], Loss: 1171.5153
Epoch [140/200], Loss: 1171.4670
Epoch [145/200], Loss: 1171.3969
Epoch [150/200], Loss: 1171.3367
Epoch [155/200], Loss: 1171.28

In [3778]:
outputs=model(inputs)
rss = ((targets - outputs)**2).sum()
MSE = rss/len(targets)
print("FINAL MSE:")
print(MSE)
# for i in range(0,10):
#     prediction = outputs[i] 
#     label = targets[i]
    
#     print("prediction = {0},  label = {1}".format(prediction,label)) 

FINAL MSE:
tensor(1171.4452)
